In [51]:
import pandas as pd
import numpy as np
import math

In [52]:
directions = ["se-no", "ne-so", "no-se", "so-ne", "e-o", "o-e", "n-s" ]
def remove_directions(x: str):
    for s in directions:
        x = x.replace(s, "")
        x = x.strip()
    return x

print(remove_directions("33 avenue des champs elysées no-se"))

33 avenue des champs elysées


In [56]:
link_streets_compteurs = pd.read_csv(".\\..\dataset\\accident_flux.csv", sep = ",")
link_streets_compteurs = link_streets_compteurs [["voie","Nom du site de comptage","nom_compteur"]]

link_streets_compteurs = link_streets_compteurs.drop_duplicates()

link_streets_compteurs = link_streets_compteurs.sort_values(by = ["voie", "Nom du site de comptage"])

array_link = np.array(link_streets_compteurs)
new_array_link = []
for i in range(len(array_link)):
    for j in range(1,3):
        if type(array_link[i, j]) == str:
            new_array_link.append([array_link[i, 0], array_link[i, j]]) 

new_array_link = np.array(new_array_link)
new_array_link

link_streets_compteurs = pd.DataFrame(data = new_array_link, columns = ["voie",	"Nom du site de comptage"])
link_streets_compteurs["Nom du site de comptage"] = link_streets_compteurs["Nom du site de comptage"].apply(lambda x: str.lower(x))
link_streets_compteurs = link_streets_compteurs.drop_duplicates()

link_streets_compteurs["Nom du site de comptage"] = link_streets_compteurs["Nom du site de comptage"].apply(remove_directions)
link_streets_compteurs = link_streets_compteurs.drop_duplicates()
link_streets_compteurs["name"] = link_streets_compteurs["Nom du site de comptage"]
link_streets_compteurs = link_streets_compteurs.drop(["Nom du site de comptage"], axis = 1)
link_streets_compteurs.index = range(len(link_streets_compteurs.index))

link_streets_compteurs


,voie,name
0,avenuedaumesnil,135 avenue daumesnil
1,avenuedeflandre,87 avenue de flandre
2,avenuedelagrandearmee,10 avenue de la grande armée
3,avenuedelagrandearmee,7 avenue de la grande armée
4,avenuedenfertrochereau,106 avenue denfert rochereau
5,avenuedenfertrochereau,97 avenue denfert rochereau
6,avenuedeschampselysees,33 avenue des champs elysées
7,avenuedeschampselysees,44 avenue des champs elysées
8,boulevardaugusteblanqui,21 boulevard auguste blanqui
9,boulevarddemagenta,89 boulevard de magenta


In [58]:
print(len(link_streets_compteurs["voie"].value_counts()))


18


In [60]:
flux = pd.read_csv(".\\..\dataset\\traindata_flux_compteurs_sorted.csv", sep = ",")
flux = flux.drop(["Unnamed: 0"], axis = 1)
flux["mois_annee_comptage"] = flux["mois_annee_comptage"].apply(lambda x : str(x[:-3]))
flux["name"] = flux["name"].apply(lambda x : str.lower(x))

flux["name"] = flux["name"].apply(remove_directions)


In [61]:
len(flux.name.unique())

88

In [73]:
flux

,name,mois_annee_comptage,sum_counts
0,10 avenue de la grande armée,2018-07,1382.0
1,10 avenue de la grande armée,2018-08,8030.0
2,10 avenue de la grande armée,2018-09,12009.0
3,10 avenue de la grande armée,2018-10,10184.0
4,10 avenue de la grande armée,2018-11,8203.0
...,...,...,...
3222,face au 8 avenue de la porte de charenton,2019-12,48849.0
3223,pont des invalides,2019-11,9942.0
3224,pont des invalides,2019-12,23628.0
3225,pont des invalides s-n,2019-11,10932.0


In [76]:

flux_no_directions = flux.groupby(["name", "mois_annee_comptage"])["sum_counts"].mean().reset_index()

flux_no_directions


,name,mois_annee_comptage,sum_counts
0,10 avenue de la grande armée,2018-07,1382.0
1,10 avenue de la grande armée,2018-08,8030.0
2,10 avenue de la grande armée,2018-09,12009.0
3,10 avenue de la grande armée,2018-10,10184.0
4,10 avenue de la grande armée,2018-11,8203.0
...,...,...,...
3076,voie georges pompidou,2022-09,108777.0
3077,voie georges pompidou,2022-10,104891.0
3078,voie georges pompidou,2022-11,88354.0
3079,voie georges pompidou,2022-12,53005.0


In [85]:
df = pd.merge(left = flux_no_directions, right = link_streets_compteurs, how = 'left', on = 'name')


df["date"] = df["mois_annee_comptage"]
df = df.drop(["mois_annee_comptage"], axis = 1)

df = df.groupby(["voie", "date"])["sum_counts"].mean().reset_index()

df


,voie,date,sum_counts
0,avenuedaumesnil,2018-01,18903.0
1,avenuedaumesnil,2018-02,14251.0
2,avenuedaumesnil,2018-03,21653.0
3,avenuedaumesnil,2018-04,29096.0
4,avenuedaumesnil,2018-05,33336.0
...,...,...,...
863,voiegeorgespompidou,2022-09,108777.0
864,voiegeorgespompidou,2022-10,104891.0
865,voiegeorgespompidou,2022-11,88354.0
866,voiegeorgespompidou,2022-12,53005.0


In [48]:
len(df.name.unique())

25

In [91]:
accidents = pd.read_csv(".\\..\dataset\\accident_by_month_with_category.csv", sep = ",")
accidents = accidents.drop(["Unnamed: 0"], axis = 1)

accidents[accidents["voie"] == "avenuedaumesnil"]

,voie,ame_d,date,n_accidents
144,avenuedaumesnil,VOIE SECURISEE,2021-06,0
145,avenuedaumesnil,VOIE SECURISEE,2021-05,0
146,avenuedaumesnil,VOIE SECURISEE,2019-05,0
147,avenuedaumesnil,VOIE SECURISEE,2020-12,0
148,avenuedaumesnil,VOIE SECURISEE,2019-09,1
149,avenuedaumesnil,VOIE SECURISEE,2019-10,1
150,avenuedaumesnil,VOIE SECURISEE,2020-03,0
151,avenuedaumesnil,VOIE SECURISEE,2021-07,0
152,avenuedaumesnil,VOIE SECURISEE,2019-08,0
153,avenuedaumesnil,VOIE SECURISEE,2019-06,0


In [95]:
df_final = pd.merge(left = df, right = accidents, how = 'left', on = ['voie', 'date'])

df_final = df_final.dropna()

df_final["flux"] = df_final["sum_counts"]
df_final = df_final.drop(["sum_counts"], axis = 1)

df_final["categorie"] = df_final["ame_d"]
df_final = df_final.drop(["ame_d"], axis = 1)

df_final

,voie,date,n_accidents,flux,categorie
12,avenuedaumesnil,2019-01,1.0,27243.0,VOIE SECURISEE
13,avenuedaumesnil,2019-02,0.0,29996.0,VOIE SECURISEE
14,avenuedaumesnil,2019-03,0.0,32998.0,VOIE SECURISEE
15,avenuedaumesnil,2019-04,1.0,37451.0,VOIE SECURISEE
16,avenuedaumesnil,2019-05,0.0,36827.0,VOIE SECURISEE
...,...,...,...,...,...
850,voiegeorgespompidou,2021-08,0.0,58728.0,MOINS SECURISEE
851,voiegeorgespompidou,2021-09,0.0,111075.0,MOINS SECURISEE
852,voiegeorgespompidou,2021-10,0.0,94710.0,MOINS SECURISEE
853,voiegeorgespompidou,2021-11,0.0,74473.0,MOINS SECURISEE


In [96]:
df_final.to_csv('.\\..\dataset\\accidents_date_voie_flux_cat.csv',  index = False)